In [24]:
# -*- coding: utf-8 -*-
# This code is modified from Mizuuchi et al 2022: https://github.com/rmizuuchi/Mizuuchi_Evo_RNA_Complexity
# See 'defining parameters' section and table S1 of supplementary information for the meaning and values of all parameters
# Please install the required packages before running.
# (Last modified: 24Apr, 2023 by Ming Liu)
# Loading packages
import numpy as np
from matplotlib import pyplot as plt
from math import *
import seaborn as sns
from scipy.integrate import odeint
import numba
from numba import jit
import time
import random
## For deprecation warnings
from numba import NumbaDeprecationWarning, NumbaPendingDeprecationWarning
import warnings
warnings.simplefilter('ignore', category=NumbaDeprecationWarning)
warnings.simplefilter('ignore', category=NumbaPendingDeprecationWarning)

# Replication (The ODE function)
@jit #faster than Cython 
def replication(v0, t, Cn, a, b):
    dv = np.empty_like(v0)
    Capacity = (1-(v0[0]+v0[1])/Cn)
    Grate= [2.,2.]
    dv[0]= Grate[0]*v0[0]*Capacity
    if(np.sum(v0)> 0):
        dv[1]= Grate[1]*v0[1]*4.0*(1-a+a/(1+np.exp(-3*(v0[0]-1.5))))*(1-b+b/(1+np.exp(-10*(v0[0]/(np.sum(v0))-0.5))))*Capacity
    else:
        dv[1]= 0.
    return dv

# Initial concentrations
def initial(M, Coop_i, Cheat_i):
    comps = np.vstack([np.random.poisson(popn/float(M),M) for popn in [Coop_i, Cheat_i]]) 
    return comps

# Dilution (Diluted D times and convert densities to integer)
def dilution(D,M,comps):
    np.random.shuffle(comps.T) # Randomization
    comps= comps[:,:round(M/D)] # Dilution
    newcomps = np.vstack([np.zeros(M-round(M/D)) for i in range(2)]) # New compartments
    comps = np.hstack([comps, newcomps]) #Combine
    return comps

# Repartition (Fusion-division of two randomly picked compartments)
def repartition(Fnumber, comps):
    X = list(range(M))
    ids = [random.sample(X, 2) for i in range(Fnumber)] #randomly choose two compartments for M*F times
    for i in ids:
        p,q = i[0],i[1]
        if np.sum(comps[:,p]+comps[:,q])>0: # Other than two vacant compartments
            fuse = (comps[:,p] + comps[:,q]).astype(np.int32) # Fusion of two compartments
            comps[0,p], comps[1,p] = \
                np.random.binomial(fuse[0],0.5),np.random.binomial(fuse[1],0.5) # Re-division; density is treated as interger
            comps[:,q] = fuse - comps[:,p]
    return comps

# Defining parameters 
Round= 200  # Length of the simulation
M= 4000     # Number of subpopulations
Cn= 100     # Carrying capacity of each subpopulation
D= 5        # Dilution ratio
Fnumber=int(M*1.3) # M*F in the paper; the number of repeated mixing processes
IDs = np.array(['Coop','Cheat']) # Each corresponds to cooperators and cheats
a= 1        # Wrighting of density dependence
b= 0        # Weighting of frequency dependence
# Temporal settings
reaction_time= 4
N_dots= 10
t = np.linspace(0, reaction_time, N_dots)
temp= np.empty([M, N_dots, 2])
# Initialisation
[Coop_i,Cheat_i]=[M,M]
Coop_s, Cheat_s= [Coop_i], [Cheat_i]
concs = [Coop_s, Cheat_s]
comps= initial(M,Coop_i,Cheat_i)
R_true = 0
# Main simulation codes
for R in range(Round):
    R_true+=1
    if np.sum(comps)>0: # Only if a replicator is remaining
        if R != 0:
            comps = dilution(D,M,comps) #dilution
            comps = repartition(Fnumber, comps) #repartition
            active_comp_IDs = np.where(np.sum(comps,axis=0)>0)[0] # IDs of subpopulations containing some individuals
            comps = np.hstack([comps[:,active_comp_IDs]]) # Subsetting 
        # Replication (population growth)
        for k in range(len(comps[0])):
            RNAs = np.hstack([comps[j][k] for j in range(2)])
            v =  odeint(replication, RNAs, t, args=(Cn,))
            ## Within growth cycle recording
            for i in range(v.shape[0]):
                temp[k, i, 0]= v[i, 0]
                temp[k, i, 1]= v[i, 1]
            ## Getting the population size at the end of focal growth cycle
            for l in range(2):
                comps[l][k]=round(v[-1][l])
        # Return vacant subpopulations
        new = M-len(comps[0])
        comps = np.hstack([comps,np.zeros(new*2).reshape(2,new)])
    for j in range(v.shape[0]):
        for i in range(2):
            concs[i].append(np.sum(temp[:,j,i])) 
# Saving the output
txt_file= open("output.txt", "w")
txt_file.write("time\tN1\tN2\n") # N1= cooperators, N2= cheats
for i in range(np.size(concs[0])):
    txt_file.write("%f\t%f\t%f\n" % (i/N_dots*reaction_time, concs[0][i]/M, concs[1][i]/M))
txt_file.close()